In [3]:
#hybrid model
from nltk.tokenize import word_tokenize, ToktokTokenizer
from nltk import ngrams
from nltk.metrics.distance import edit_distance
from pandas import DataFrame, read_csv, read_table
import os
from joblib import Parallel, delayed
from collections import defaultdict

## Word-level N-Grams predicting + words distance

In [18]:
PATH = "../Corpora/TO_CREATE_VALID_UTTERANCE/GOOGLE_1_billion_word/heldout"

In [46]:
def get_ngrams(PATH, n):
    bi_grams = defaultdict(int)
    for filename in os.listdir(PATH):
        with open(os.path.join(PATH, filename), "r") as f:
            for line in f:
                bi = ngrams(word_tokenize(line), n)
                for token in bi:
                    bi_grams[token] += 1
       
    return bi_grams


In [47]:
gramed = get_ngrams(PATH, 3)

In [ ]:
sorted(a, key=lambda x: a[x], reverse=True)

In [45]:
len(a)

4749409

In [ ]:
def get_top(tab, p, c):
    if len(tab) == 0:
        return tab.append((c,p))
    low = len(tab)-1
    top = 0
    while top < low:
        pom = (low-top) // 2
        if tab[pom][1] > p:
            top = pom
        else:
            low = pom
    return tab[: low] [(c,p)] + tab[low: ]

In [ ]:
# wykorzystaj rożne metryki, edit_distance, cosinusowe_z_embeddingów, jaccard, Jaro-Winkler distance
def get_closest(tokens, token):
    best_token = tokens[0]
    min_dist = edit_distance(best_token, token)
    for t in tokens[1: ]:
        if edit_distance(t, token) < min_dist:
            min_dist = edit_distance(t, token)
            best_token = t
    print(best_token, min_dist)
    return best_token

In [ ]:
#sprawdz jeszcze z opcją by w topce umieszczać tokeny zaczynające się lub kończące na tą samą literę
# zwiększ topke, może top100
#popraw ostatni krok z joinem
def find_and_check(sentence, glossary, grams, n):
    approx_sentence = []
    gramed_sentence = list(ngrams(word_tokenize(sentence), n))
    for token1, token2, token3 in gramed_sentence:
#         sub_grams = [gram for gram in grams if gram[0] == token1 and gram[1] == token2]
        sub_grams = [gram for gram in grams if g]
        top10 = ['']
        for c in glossary:
            p = sum([sub_grams[gram] for gram in sub_grams if gram[3] == c]) / sum([sub_grams[gram] for gram in sub_grams])
            if top10[0][n-1] < p:
                top10 = get_top(top10, p, c)
        tokens, dist = list(zip(*top10))
        approx = get_closest(tokens, token3)
        approx_sentence.append(approx)
    return " ".join(approx_sentence)

In [48]:
PATH = "../N-grams"
df = read_csv(os.path.join(PATH, "valid_utterances1.csv"))
# valid = list(df["valid"].values)
# df = DataFrame(valid)
df.columns = ["valid"]

In [ ]:
df = df[:100000]
data = ["I am suppoed too just fel like he is sincere"]

In [ ]:
grams, glossary = create_ngrams_from_text(df, 3)

In [ ]:
approx_data = Parallel(n_jobs=-1)(delayed(find_and_check)(sentence, words, gramed, 3) for sentence in data)

In [55]:
from nltk.corpus import words

In [56]:
words.words

<bound method WordListCorpusReader.words of <WordListCorpusReader in '/home/user/nltk_data/corpora/words'>>

In [58]:
PATH = "../N-grams/"


In [59]:
british_gloss = read_csv(PATH + "british-english.csv", header=None)
british_gloss.columns = ["word"]
usa_gloss = read_csv(PATH + "american-english.csv", header=None)
usa_gloss.columns = ["word"]

In [61]:
words = list(british_gloss.word.unique()) + list(",./';[]<>?:{}!@#$%^&*()_+-=") + list(usa_gloss.word.unique())
words = list(set(words))